# Exercise: Eco-Friendly Product Launch
From: https://estudijas.rtu.lv/mod/assign/view.php?id=4116908

## Background:

Imagine you're part of a European eco-friendly startup, Green Innovate EU, focusing on launching sustainable products. Your team is ready to introduce two new items: a biodegradable water bottle BIO_BOT and a solar-powered phone charger SOL_CHA. Both products align with the company's mission to reduce plastic waste and promote renewable energy sources. The challenge lies in maximizing profit while adhering to budget constraints and production capabilities.

Problem Statement:

Green Innovate EU has a budget of €50,000 for the first production run. The cost to produce one biodegradable water bottle is €3, and the cost for one solar-powered phone charger is €8. The expected profit from each water bottle is €5, and from each charger is €12. The market research team has indicated that due to the current market trends and consumer demand, the company should produce at least twice as many water bottles as chargers. Moreover, the production facility's constraints allow for the manufacture of a maximum of 10,000 water bottles and 4,000 chargers for this run.

Objectives:

Determine how many of each product the startup should produce to maximize profit.
Ensure that the production plan respects the budget limit, production capacity, and market demand insights.
Requirements:

Let 
�
1
 represent the number of biodegradable water bottles to produce.
Let 
�
2
 represent the number of solar-powered phone chargers to produce.
Maximize the total profit 
�
=
5
�
1
+
12
�
2

Subject to:
Budget constraint: 
3
�
1
+
8
�
2
≤
50
,
000

Production constraints: 
�
1
≤
10
,
000
, 
, x
2
≤
4
,
000

Market demand constraint: 
�
1
≥
2
�
2

Task:

Use linear programming techniques to find the optimal production quantities of water bottles and chargers. Illustrate your solution process and discuss the implications of your results for the startup's strategy.



Allowed approaches:

While it would be extremely impressive, if you could implement a simplex algorithm from scratch, you are allowed to use libraries for this solution.
Also you are allowed to try different approach, even some sort of brute-force solution.
Also you are allowed to use pen and paper to solve this problem by hand and then verify solution.
An interesting attempt would be to use some sort of hill-climbing approach.

In [1]:
# check if we have pulp if not we install it
try:
    import pulp
except ImportError:
    print("pulp not installed")
    # import pip
    # pip.main(['install', 'pulp'])
    # import pulp

In [3]:
!pip install pulp

Defaulting to user installation because normal site-packages is not writeable
  Obtaining dependency information for pulp from https://files.pythonhosted.org/packages/09/d7/57e71e11108203039c895643368c0d1a99fe719a6a80184edf240c33d25f/PuLP-2.8.0-py3-none-any.whl.metadata
   ---------------------------------------- 17.7/17.7 MB 22.6 MB/s eta 0:00:00


  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.

[notice] A new release of pip is available: 23.2.1 -> 24.0
[notice] To update, run: C:\Program Files\Python311\python.exe -m pip install --upgrade pip


In [2]:
from pulp import *

# Create the problem variable
prob = LpProblem("Green Innovate EU", LpMaximize)

# Define the decision variables
x1 = LpVariable("BIO_BOT", lowBound=0, cat='Integer')
x2 = LpVariable("SOL_CHA", lowBound=0, cat='Integer')

# Define the objective function
prob += 5*x1 + 12*x2

# Define the constraints
prob += 3*x1 + 8*x2 <= 50000
prob += x1 <= 10000
prob += x2 <= 4000
prob += x1 >= 2*x2

# Solve the problem
prob.solve()

# Print the optimal solution
print("Optimal production quantities:")
print("BIO_BOT:", value(x1))
print("SOL_CHA:", value(x2))
print("Total profit:", value(prob.objective))

Optimal production quantities:
BIO_BOT: 10000.0
SOL_CHA: 2500.0
Total profit: 80000.0


C:\Users\val-wd\AppData\Roaming\Python\Python311\site-packages\pulp\pulp.py:1316: UserWarning: Spaces are not permitted in the name. Converted to '_'
  warnings.warn("Spaces are not permitted in the name. Converted to '_'")


## Brute force approach

In [3]:
# let's try to find max between 0 and 10000 for x1 and 0 and 4000 for x2
max_profit = 0
for x1 in range(0, 10001):
    for x2 in range(0, 4001):
        if 3*x1 + 8*x2 <= 50000 and x1 >= 2*x2:
            profit = 5*x1 + 12*x2
            if profit > max_profit:
                max_profit = profit
                max_x1 = x1
                max_x2 = x2
print("Optimal production quantities:")
print("BIO_BOT:", max_x1)
print("SOL_CHA:", max_x2)
print("Total profit:", max_profit)

Optimal production quantities:
BIO_BOT: 10000
SOL_CHA: 2500
Total profit: 80000


## Hill-Climbing approach



In [4]:
# we can start moving quicker by making bigger steps in the search space initially
# because we have linear constraints we do not expect to see any valleys or peaks
max_profit = 0
for x1 in range(0, 10001, 100):
    for x2 in range(0, 4001, 100):
        if 3*x1 + 8*x2 <= 50000 and x1 >= 2*x2:
            profit = 5*x1 + 12*x2
            if profit > max_profit:
                max_profit = profit
                max_x1 = x1
                max_x2 = x2

print("Optimal production quantities:")
print("BIO_BOT:", max_x1)
print("SOL_CHA:", max_x2)
print("Total profit:", max_profit)

Optimal production quantities:
BIO_BOT: 10000
SOL_CHA: 2500
Total profit: 80000


In [9]:
# That worked but it is possible that we missed the optimal solution if it was in the middle of the step
# idea is to make the step adjustable as we get closer to the solution
# our minimal steps are 1, 1
# we can start with 100, 100
# if we are close to the solution we can reduce the step to 1, 1
# we can use increase in profit as a measure of how close we are to the solution
max_profit = 0
step = 100
profit_increase = 0

# lets caclucate potential profit if we relax all constraints except keep number of units

x1_cand = 10_000
x2_cand = 4_000
potential_profit = 5*x1_cand + 12*x2_cand
steps = []
for x1 in range(0, 10001, step):
    for x2 in range(0, 4001, step):
        if 3*x1 + 8*x2 <= 50000 and x1 >= 2*x2:
            profit = 5*x1 + 12*x2
            if profit > max_profit:
                # profit_increase = profit - max_profit   
                max_profit = profit
                max_x1 = x1
                max_x2 = x2
                # now we could adjust step based on profit_increase
                # ideas as profit increase get smaller we can reduce the step
                # now we could adjust step based on how close our max_profit is to potential_profit
                # if we are close we can reduce the step
                # if we are far we can increase the step
                # TODO our new_step should be based on how close we are to the potential_profit
                # currently it is very ad hoc and not based on any theory...
                # notably 10_000 is a magic number at the moment
                new_step = max(min(100, int((potential_profit - max_profit)/10_000)), 1)
                if new_step != step:
                    step = new_step
                    steps.append(step)
print("Optimal production quantities:")
print("BIO_BOT:", max_x1)
print("SOL_CHA:", max_x2)
print("Total profit:", max_profit)

Optimal production quantities:
BIO_BOT: 10000
SOL_CHA: 2500
Total profit: 80000


In [10]:
len(steps)

9

In [11]:
steps

[9, 8, 7, 6, 5, 4, 3, 2, 1]

## Ideas - Branch and bound approach

We relax the requirement that the number of water bottles must be an integer. We can then solve the problem using the simplex algorithm. We then round the solution to the nearest integer and check if the solution is feasible. If it is, we check if the solution is better than the best solution found so far. If it is, we update the best solution. We then branch on the variable that is not an integer and repeat the process. We stop when we have checked all branches or when we can prove that the best solution found so far is optimal.
